## QCLab: Deutsch–Jozsa Algorithm – Constant or Balanced?

The Deutsch–Jozsa algorithm was the first quantum algorithm to show an **exponential separation** (speedup) between quantum and classical computation. It solves the problem of determining whether a Boolean function $f:\{0,1\}^n \to \{0,1\}$ is constant** or balanced.

- A function $f : \{0,1\}^n \to \{0,1\}$ is:
  - **constant** if it returns the same output (always 0 or always 1) for all inputs.
  - **balanced** if it returns 0 for exactly half of the inputs, and 1 for the other half.

Classically, in the worst case one needs to evaluate the function $f$ on $2^{n-1}+1$ different inputs to be certain whether $f$ is constant or balanced.  
The quantum version solves the problem with **just one evaluation** of the oracle by exploiting superposition and interference.  


The Deutsch–Jozsa algorithm (shown in the figure) prepares the input qubits in superposition using Hadamard gates, applies the oracle $O_f$ that encodes the function $f$, and then uses another layer of Hadamards to create interference.  
Measuring the first $n$ qubits reveals whether $f$ is constant (all 0s) or balanced (at least one qubit measured as 1).


![Deutsch–Jozsa](images/Deutsch–Jozsa.png)

---

### Taks

Construct a Deutsch–Jozsa quantum circuit to determine whether the Boolean function

$$
f(x_0, x_1, x_2, x_3) = (x_0 \land x_1) \oplus (x_2 \oplus x_3)
$$

is constant or balanced, using a single quantum query. The function should be implemented as a quantum oracle within the circuit.

### Expected Output
- A plot of the Deutsch–Jozsa circuit.  
- A histogram of the measurement results.  
- A printout of the measurement results dictionary.  
- A message indicating the function type and the algorithm’s finding: constant or balanced.  

### Experimentation

- Try running the Deutsch–Jozsa algorithm with different oracles that implement constant and balanced functions.  
- Compare the measurement histograms: constant functions should always yield the all-zero state, while balanced functions will produce non-zero bitstrings.  
- Verify that, regardless of $n$, the quantum algorithm needs only one oracle call to distinguish the two cases.  
- Modify the circuit to use even more input qubits and observe how the results scale.  


In [ ]:
# ====================================================
# QCLab: Deutsch–Jozsa Algorithm
# <QC|CT> qcict.org
# ====================================================

from IPython.display import display

from qiskit import QuantumCircuit, QuantumRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram,plot_distribution
from qiskit.visualization import circuit_drawer

def deutsch_jozsa_oracle():
    """
    Oracle for the Boolean function:
        f(x0, x1, x2, x3) = (x0 AND x1) XOR (x2 XOR x3)

    The oracle implements the transformation:
        |x⟩|y⟩ → |x⟩|y ⊕ f(x)⟩

    In Boolean algebra, XOR is associative and commutative, so:
        (x0 AND x1) XOR (x2 XOR x3)
    can be rewritten as:
        ((x0 AND x1) XOR x2) XOR x3
    This form matches the implementation order in the code, where the target
    qubit `f` is first updated with (x0 AND x1), then XORed with x2, and finally
    XORed with x3.

    Barriers are included for visual clarity in circuit diagrams, separating
    the oracle from other steps in the Deutsch–Jozsa algorithm.
    """
    x = QuantumRegister(4, name='x')
    f = QuantumRegister(1, name='f')
    qc = QuantumCircuit(x, f, name="Oracle")

    qc.barrier()  # visual separation from previous circuit steps

    # (x0 AND x1) → XOR into target qubit f
    qc.ccx(x[0], x[1], f[0])

    # XOR x2 into target qubit
    qc.cx(x[2], f[0])

    # XOR x3 into target qubit
    qc.cx(x[3], f[0])

    qc.barrier()  # visual separation from subsequent circuit steps

    return qc

def deutsch_jozsa_circuit():
    """
    Builds the Deutsch–Jozsa circuit for 4 input qubits and 1 target qubit.

    The circuit applies Hadamard gates to all input qubits, prepares the target
    qubit in the |−⟩ state, inserts the oracle between two layers of Hadamards
    on the input qubits, and measures the result to determine whether the 
    function is constant or balanced.

    Returns:
        QuantumCircuit: The complete 5-qubit Deutsch–Jozsa circuit with measurements.
    """
    qc = QuantumCircuit(5, 4)

    # Step 1: Initialize input qubits to superposition
    for i in range(4):
        qc.h(i)

    # Step 2: Prepare target qubit in |−⟩ state
    qc.x(4)
    qc.h(4)

    # Step 3: Apply oracle
    oracle = deutsch_jozsa_oracle()
    qc.compose(oracle, inplace=True)

    # Step 4: Apply Hadamard to input qubits again
    for i in range(4):
        qc.h(i)

    # Step 5: Measure input qubits
    qc.measure(range(4), range(4))

    return qc

# ------------------------------------------------
#                main program
# ------------------------------------------------

# -- build and run circuit --
qc = deutsch_jozsa_circuit()
simulator = AerSimulator()
result = simulator.run(qc, shots=1000).result()
counts = result.get_counts(qc)

# -- display --
display(circuit_drawer(qc, style="bw", output="mpl"))
display(plot_histogram(counts))

# Check if '000' is among the measurement outcomes
print("Measurement results:", counts)
if '000' in counts:
    print("The function f(x0, x1, x2, x3) = (x0 AND x1) XOR (x2 XOR x3) is CONSTANT (found '000' in results).")
else:
    print("The function f(x0, x1, x2, x3) = (x0 AND x1) XOR (x2 XOR x3) is BALANCED (no  '000' in results).")
    